## Requirements


In [1]:
# Unable warnings
import os
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


#### Enable GPU processing

In [2]:
from numba import cuda 
cuda.get_current_device().reset()
cuda.current_context().reset()
cuda.current_context().get_memory_info()

MemoryInfo(free=828375040, total=6233522176)

In [2]:
import keras.backend as K
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        K.clear_session()
        K._get_available_gpus()
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)


#### Directory adjustment


In [ ]:
from pathlib import Path
import sys

# Back to main folder
path = os.path.dirname(os.getcwd()) + "/"
os.chdir(path)
sys.path.append(path)


#### Data Processing


In [ ]:
# ETL
import numpy as np
import pandas as pd
import multiprocessing


#### Natural language processing


In [ ]:
from gensim.models import KeyedVectors
from gensim import models
import gensim


#### Models

In [ ]:
# ML preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split


# Deep learnig model
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional


In [ ]:
# Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# Train  metrics
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]


## Split dataset


In [ ]:
# Get data
df = pd.read_csv("data/corpus/augmented_corpus_fortuna.csv")

# Set target and features
target = "label"
features = "text_nonstop"
count = f"length_{features}"
pos = len(df.query('label==1'))
neg = len(df.query('label==0'))


# Break apart dataset
X = df[features].values.astype("U")
y = df[target]

# Split train abd test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Set k-fold criteria
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

# Classes balancing
longest_text = df[count].max()
initial_bias = np.log([pos/neg])

weight_for_0 = (1 / neg) * (len(df) / 2.0)
weight_for_1 = (1 / pos) * (len(df) / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}


## Neural network

In [ ]:
# Preset parameters
experiment_parameters = {"classifier": "LSTM",
                         "class_weight": class_weight,
                         "epochs": 200,
                         "units": 50,
                         "dropout": 0.4,
                         "recurrent_dropout": 0.2,
                         "kernel_initializer": 'glorot_uniform',
                         "loss": "binary_crossentropy",
                         "optimizer": "adamax",
                         "batch_size": 64}


#### Basic pipeline

In [ ]:
from keras.layers import TextVectorization
from keras.layers import Input


In [ ]:
tfidf_vectorizer = TextVectorization(max_tokens=1500, standardize="lower_and_strip_punctuation",
                                     split="whitespace", output_mode="tf-idf", ngrams=(1, 2))
tfidf_vectorizer.adapt(X_train, batch_size=32)
vocab = tfidf_vectorizer.get_vocabulary()
out = tfidf_vectorizer(X_train[:5])


In [ ]:
# Simple Model
def nn_builder(text_vectorizer):
    nn = Sequential()
    nn.add(Input(shape=(1,), dtype="string"))
    nn.add(text_vectorizer)
    nn.add(Dense(500, activation="relu"))
    nn.add(Dropout(0.4))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(loss=experiment_parameters['loss'],
               optimizer=experiment_parameters['optimizer'],
               metrics=METRICS)
    return nn


nn_model = nn_builder(tfidf_vectorizer)

history = nn_model.fit(X_train, y_train,
                       experiment_parameters['batch_size'],
                       experiment_parameters['epochs'],
                       validation_data=(X_test, y_test),
                       callbacks=[EarlyStopping(monitor="loss",
                                                patience=10,
                                                restore_best_weights=True)])


Epoch 1/200
71/71 [==============================] - 2s 12ms/step - loss: 0.8860 - tp: 464.0000 - fp: 789.0000 - tn: 2318.0000 - fn: 965.0000 - accuracy: 0.6133 - precision: 0.3703 - recall: 0.3247 - auc: 0.5204 - prc: 0.3431 - val_loss: 0.5791 - val_tp: 91.0000 - val_fp: 57.0000 - val_tn: 720.0000 - val_fn: 266.0000 - val_accuracy: 0.7152 - val_precision: 0.6149 - val_recall: 0.2549 - val_auc: 0.6737 - val_prc: 0.5296
Epoch 2/200
71/71 [==============================] - 0s 5ms/step - loss: 0.7021 - tp: 588.0000 - fp: 610.0000 - tn: 2497.0000 - fn: 841.0000 - accuracy: 0.6801 - precision: 0.4908 - recall: 0.4115 - auc: 0.6358 - prc: 0.4513 - val_loss: 0.5558 - val_tp: 130.0000 - val_fp: 67.0000 - val_tn: 710.0000 - val_fn: 227.0000 - val_accuracy: 0.7407 - val_precision: 0.6599 - val_recall: 0.3641 - val_auc: 0.7170 - val_prc: 0.5733
Epoch 3/200
71/71 [==============================] - 0s 5ms/step - loss: 0.6070 - tp: 673.0000 - fp: 506.0000 - tn: 2601.0000 - fn: 756.0000 - accuracy: 0

In [ ]:
test_preds = nn_model.predict(X_test)
pd.DataFrame(classification_report(
    y_test, np.where(test_preds >= 0.5, 1, 0), output_dict=True))


36/36 [==============================] - 0s 2ms/step


,0,1,accuracy,macro avg,weighted avg
precision,0.775155,0.534954,0.705467,0.655055,0.699536
recall,0.803089,0.492997,0.705467,0.648043,0.705467
f1-score,0.788875,0.513120,0.705467,0.650997,0.702063
support,777.000000,357.000000,0.705467,1134.000000,1134.000000


## Deep learning

#### Basic pipeline

In [ ]:
# Deep learnig Model
def nn_builder(text_vectorizer):
    nn = Sequential()
    nn.add(Input(shape=(1,), dtype="string"))
    nn.add(text_vectorizer)
    nn.add(Dense(1500, activation="relu"))
    nn.add(Dense(500, activation="relu"))
    nn.add(Dropout(0.40))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(loss=experiment_parameters['loss'],
               optimizer=experiment_parameters['optimizer'],
               metrics=METRICS)
    return nn


deep_model = nn_builder(tfidf_vectorizer)

history = deep_model.fit(X_train, y_train,
                         experiment_parameters['batch_size'],
                         experiment_parameters['epochs'],
                         validation_data=(X_test, y_test))


Epoch 1/200
71/71 [==============================] - 2s 12ms/step - loss: 1.0905 - tp: 548.0000 - fp: 775.0000 - tn: 3109.0000 - fn: 1238.0000 - accuracy: 0.6450 - precision: 0.4142 - recall: 0.3068 - auc: 0.5674 - prc: 0.3777 - val_loss: 0.6486 - val_tp: 231.0000 - val_fp: 294.0000 - val_tn: 483.0000 - val_fn: 126.0000 - val_accuracy: 0.6296 - val_precision: 0.4400 - val_recall: 0.6471 - val_auc: 0.6933 - val_prc: 0.5217
Epoch 2/200
71/71 [==============================] - 0s 6ms/step - loss: 0.5655 - tp: 472.0000 - fp: 274.0000 - tn: 2833.0000 - fn: 957.0000 - accuracy: 0.7286 - precision: 0.6327 - recall: 0.3303 - auc: 0.7125 - prc: 0.5500 - val_loss: 0.5684 - val_tp: 100.0000 - val_fp: 35.0000 - val_tn: 742.0000 - val_fn: 257.0000 - val_accuracy: 0.7425 - val_precision: 0.7407 - val_recall: 0.2801 - val_auc: 0.6949 - val_prc: 0.5529
Epoch 3/200
71/71 [==============================] - 0s 5ms/step - loss: 0.4920 - tp: 653.0000 - fp: 249.0000 - tn: 2858.0000 - fn: 776.0000 - accuracy

In [ ]:
test_preds = deep_model.predict(X_test)
pd.DataFrame(classification_report(
    y_test, np.where(test_preds >= 0.5, 1, 0), output_dict=True))


36/36 [==============================] - 0s 2ms/step


,0,1,accuracy,macro avg,weighted avg
precision,0.756396,0.587234,0.72134,0.671815,0.703141
recall,0.875161,0.386555,0.72134,0.630858,0.721340
f1-score,0.811456,0.466216,0.72134,0.638836,0.702769
support,777.000000,357.000000,0.72134,1134.000000,1134.000000
